<a href="https://colab.research.google.com/github/xiaorui777/NLP/blob/master/Build_Search_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build Search Engine

#### Input：Words
#### Output： Documents

In [0]:
csv_path = './sqlResult_1558435.csv'

In [0]:
import pandas as pd
csv = pd.read_csv(csv_path, encoding = 'gb18030')

In [0]:
content = csv.fillna(' ')
news_content = content['content'].tolist()

In [0]:
# 中文分词
import jieba
def cut(string): return ' '.join(jieba.cut(string))

In [0]:
# 切词
import re
def token(string):
    return re.findall(r'[\d|\w]+', string)

In [0]:
news_content = [token(n) for n in news_content]

In [7]:
news_content = [' '.join(n) for n in news_content]
news_content = [cut(n) for n in news_content]

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.013 seconds.
Prefix dict has been built succesfully.


In [8]:
news_content[2]

'此前 的 一加 3T 搭载 的 是 3400mAh 电池   DashCharge 快充 规格 为 5V   4A   至于 电池 缩水   可能 与 刘作 虎 所说   一加 手机 5 要 做 市面 最 轻薄 大屏 旗舰 的 设定 有关   按照 目前 掌握 的 资料   一加 手机 5 拥有 5   5 寸 1080P 三星 AMOLED 显示屏   6G   8GB   RAM   64GB   128GB   ROM   双 1600 万 摄像头   备货 量   惊喜   根据 京东 泄露 的 信息   一加 5 起 售价 是 xx99 元   应该 是 在 2799   2899   2999 中 的 某个'

## TFIDF Vectorized

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
vectorized = TfidfVectorizer()

In [0]:
sample_num = 10000
sub_samples = news_content[:sample_num]

In [0]:
X = vectorized.fit_transform(sub_samples)

In [15]:
X.shape

(10000, 135035)

# Build Search Engine
#### input: words
#### output: documents

In [0]:
def naive_search(keywords):
    news_ids = [i for i, n in enumerate(news_content) if all(w in n for w in keywords)]
# 缺点：非常耗时

In [0]:
# TFIDF的横轴是word1，word2...，纵轴是Document1，Document2...
# Boolean Search的横轴是Document1，Document2...，纵轴是word1，word2...
# 所以如果用Boolean Search优化搜索引擎的话，只需要将TFIDF转置即可

transposed_x = X.transpose().toarray()

In [27]:
word_2_id = vectorized.vocabulary_
word_2_id['此外']

83934

In [0]:
# 将 ID 当作 keys，word 当作 values
id_2_word = {d: w for w,d in word_2_id.items()}

In [28]:
id_2_word[83934]

'此外'

In [29]:
word_2_id['国际']

45792

In [32]:
word_2_id['金融']

125469

In [0]:
import numpy as np

# 在以下这些文章编号中出现了'国际'
international = set(np.where(transposed_x[45792])[0])

# 在以下这些文章编号中出现了'金融'
financial = set(np.where(transposed_x[125469])[0])

In [35]:
# 两者做一个交集，即以下的文章既出现了'国际'，又出现了'金融'
international & financial

{7,
 8,
 9,
 14,
 90,
 191,
 528,
 600,
 735,
 764,
 769,
 779,
 812,
 835,
 876,
 914,
 918,
 950,
 1028,
 1082,
 1102,
 1106,
 1125,
 1146,
 1151,
 1160,
 1201,
 1258,
 1288,
 1324,
 1395,
 1445,
 1466,
 1596,
 1599,
 1600,
 1624,
 1656,
 1745,
 1948,
 1960,
 2067,
 2099,
 2220,
 2237,
 2254,
 2314,
 2319,
 2333,
 2345,
 2356,
 2360,
 2507,
 2513,
 2542,
 2560,
 2669,
 2685,
 2696,
 2776,
 2829,
 2856,
 2861,
 2867,
 2928,
 2931,
 2960,
 3002,
 3051,
 3214,
 3461,
 3478,
 3517,
 3519,
 3573,
 3702,
 3715,
 3721,
 3740,
 3762,
 3786,
 3830,
 3837,
 3892,
 3916,
 3947,
 3953,
 3955,
 4024,
 4025,
 4083,
 4087,
 4127,
 4212,
 4235,
 4288,
 4289,
 4313,
 4318,
 4331,
 4338,
 4357,
 4361,
 4370,
 4372,
 4390,
 4407,
 4412,
 4430,
 4440,
 4457,
 4486,
 4491,
 4507,
 4582,
 4601,
 4738,
 4762,
 4831,
 4969,
 4990,
 4995,
 4999,
 5030,
 5072,
 5076,
 5085,
 5171,
 5227,
 5308,
 5380,
 5388,
 5416,
 5508,
 5514,
 5518,
 5520,
 5528,
 5537,
 5564,
 5587,
 5603,
 5606,
 5678,
 5755,
 5811,
 582

In [0]:
from functools import reduce

In [0]:
d1, d2, d3 = {1, 2, 3}, {4, 5, 6, 3, 2}, {1, 3, 4}

In [0]:
from operator import and_

In [41]:
# 求 d1, d2, d3 的交集
reduce(and_, [d1, d2, d3])

{3}

In [0]:
# 定义搜索引擎

def search_engine(query):
    '''
    @query is the searched words, splited by space
    @ return is the related documents which ranked by tfidf similarity
    '''
    words = query.split()
        
    query_vec = vectorized.transform([' '.join(words)]).toarray()[0]
    
    candidates_ids = [word_2_id[w] for w in words]
    documents_ids = [
        set(np.where(transposed_x[_id])[0]) for _id in candidates_ids
    ]
    merged_documents = reduce(and_,documents_ids)
    return merged_documents
    # 我们就能知道哪些文章包含了这些单词
    
    # 加一个文章排序
    # sorted_documents_id = sorted(merged_documents, key = lambda i: distance(query_vec, X[i].toarray()))
    # return sorted_documents_id 

In [48]:
# 而且比 naive search 快很多

search_engine('国际 金融 股价 下跌')

{1146, 4582, 4831, 5606, 6781}

In [0]:
def get_query_pat(query):
    return re.complie('({})'.format('|'.join(query_split())))

In [49]:
def search_engine_with_pretty_print(query):
    
candidates_ids = search_engine('国际 金融 股价 下跌')
for i, _id in enumerate(candidates_ids):
    print('# Search Result {}'.format(i))
    print(content['content'][_id])

# Search Result 0
“大象股”分化 短期需控仓位
　　在银行、保险等金融股的带领下，A股昨日早盘走高，上证指数冲击3200点。但盘中受到万达电影、复星医药等行业龙头股的相关传闻影响而突然“闪崩”，带动中小盘快速下跌，并拖累大盘指数尾盘跳水，沪指最终下跌0.28%，创业板再度失守1800点。
　　文、表?广州日报全媒体记者张忠安（除署名外）
　　受到周三沪深两市冲高影响，周四早盘A股一片红，上证指数最高至3186点，创业板指数也冲击到1830点附近。盘面上，家电、白酒、银行、保险、汽车等大盘权重股纷纷走高，美的集团最高上涨到43.44元，再创历史新高。贵州茅台也达到477.93元的历史峰值。另外，中国平安、海康威视、苏泊尔等纷纷在周四创出上市以来的最高位。不过，数据显示，此前强势的大盘蓝筹股出现分化，其中，保险、银行等维持强势，而白酒、家电等却放量回撤，五粮液等尾盘跳水。
　　对此，申万宏源分析师钱启敏表示，根据目前行情，一方面是大盘蓝筹股今年已经跑赢大盘，预期利好有所消化。另一方面短线市场缺乏资金支持，5月初至今存量资金净流失1700亿元，日均余额已连续8周低于1.3万亿元，成为制约行情的最重要因素。因此，他指出，A股目前短线心态严重，轮动迅速，以脉冲性走势为主，把握难度较大。
　　多股异动??机构抛售万达电影2.29亿元
　　值得注意的是，在周四A股盘面上，部分上市公司因为市场传闻而出现异动，其中，万达旗下的万达电影遭遇至少四家机构抛售，股价跌停，下午紧急停牌。而复星医药盘中也遭遇快速跳水，A股收盘跌幅超过8%，H股跌幅接近6%。
　　昨日，万达系金融产品市场表现也赚足了眼球，出现“股债”双杀，其中万达电影午后临停，截至停牌跌9.91%。同时，万达多只债券纷纷下跌。
　　有机构人士表示，万达系昨日异动主要是受到市场传闻影响。万达集团发布声明称：“网上有人恶意炒作建行等银行下发通知抛售万达债券一事，经了解建行等行从未下发此类通知，网上炒作属于谣言。”
　　记者从交易所获取的数据显示，虽然只有半天的交易，但成交量却创下今年3月1日以来的最高水平，达到12.4亿元。在股价大跌背后是机构的抛售，其中，4大机构专用席位合计卖出2.29亿元，1机构席位买入2609.27万元。申万宏源上海一营业部买入6977.41万元，南京证券一营业部卖出3435.34万元。不